In [1]:
%matplotlib inline
import pandas as pd
import numpy as np

In [2]:
cc = pd.read_csv('CatalogCrossSell.csv')

In [3]:
cc.head()

,Customer Number,Clothing Division,Housewares Division,Health Products Division,Automotive Division,Personal Electronics Division,Computers Division,Garden Division,Novelty Gift Division,Jewelry Division
0,11569,0,1,1,1,1,0,0,1,0
1,13714,0,1,1,1,1,0,1,1,1
2,46391,0,1,1,1,1,0,1,1,1
3,67264,0,0,1,1,1,0,1,1,0
4,67363,0,0,1,0,1,0,1,1,0


In [4]:
cc.shape

(4998, 10)

In [5]:
cc.isnull().sum()

Customer Number                  0
Clothing Division                0
Housewares Division              0
Health Products Division         0
Automotive Division              0
Personal Electronics Division    0
Computers Division               0
Garden Division                  0
Novelty Gift Division            0
Jewelry Division                 0
dtype: int64